In [0]:
from pyspark.sql.functions import broadcast, col, count, coalesce, lit

In [0]:
df_sales = spark.read.option('header', True).option('inferSchema', True).csv('/FileStore/tables/sales_data_set.csv')
df_stores = spark.read.option('header', True).option('inferSchema', True).csv('/FileStore/tables/stores_data_set.csv')
df_features = spark.read.option('header', True).option('inferSchema', True).csv('/FileStore/tables/Features_data_set.csv')

In [0]:
df_store_types = df_sales.join(broadcast(df_stores), on=['Store'], how='inner') \
                         .groupBy(col('Type')) \
                         .agg(count(col('Store')))
df_store_types.show(5)

+----+------------+
Type|count(Store)|
+----+------------+
 B| 163495|
 A| 215478|
 C| 42597|
+----+------------+

## Broadcast Join

In [0]:
df_store_type_a = df_stores.where(col('type') == 'A')
df_store_types = df_sales.join(broadcast(df_store_type_a), on=['Store'], how='left') \
                         .groupBy(col('Type')) \
                         .agg(count(col('Store')))
df_store_types.show(5)
df_store_types.explain()

+----+------------+
Type|count(Store)|
+----+------------+
null| 206092|
 A| 215478|
+----+------------+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[Type#2377], functions=[finalmerge_count(merge count#3424L) AS count(Store#2350)#3413L])
 +- Exchange hashpartitioning(Type#2377, 200), ENSURE_REQUIREMENTS, [id=#4853]
 +- HashAggregate(keys=[Type#2377], functions=[partial_count(Store#2350) AS count#3424L])
 +- Project [Store#2350, Type#2377]
 +- BroadcastHashJoin [Store#2350], [Store#2376], LeftOuter, BuildRight, false
 :- FileScan csv [Store#2350] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/sales_data_set.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Store:int>
 +- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [id=#4848]
 +- Filter ((isnotnull(type#2377) AND (type#2377 = A)) AND isnotnull(Store#2376))
 +- FileScan csv [Store#2376,Type#2377] Batched: false, DataFilters: [isnotnull(Type#2377), (Type#2377 = A), isnotnull(Store#2376)], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/stores_data_set.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Type), EqualTo(Type,A), IsNotNull(Store)], ReadSchema: struct<Store:int,Type:string>

## Sort Merge Join

In [0]:
df_store_types = df_sales.hint('merge').join(df_stores, on=['Store'], how='inner') \
                         .groupBy(col('Type')) \
                         .agg(count(col('Store')))
df_store_types.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[Type#2377], functions=[finalmerge_count(merge count#3504L) AS count(Store#2350)#3499L])
 +- Exchange hashpartitioning(Type#2377, 200), ENSURE_REQUIREMENTS, [id=#5093]
 +- HashAggregate(keys=[Type#2377], functions=[partial_count(Store#2350) AS count#3504L])
 +- Project [Store#2350, Type#2377]
 +- SortMergeJoin [Store#2350], [Store#2376], Inner
 :- Sort [Store#2350 ASC NULLS FIRST], false, 0
 : +- Exchange hashpartitioning(Store#2350, 200), ENSURE_REQUIREMENTS, [id=#5085]
 : +- Filter isnotnull(Store#2350)
 : +- FileScan csv [Store#2350] Batched: false, DataFilters: [isnotnull(Store#2350)], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/sales_data_set.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Store)], ReadSchema: struct<Store:int>
 +- Sort [Store#2376 ASC NULLS FIRST], false, 0
 +- Exchange hashpartitioning(Store#2376, 200), ENSURE_REQUIREMENTS, [id=#5086]
 +- Filter isnotnull(Store#2376)
 +- FileScan csv [Store#2376,Type#2377] Batched: false, DataFilters: [isnotnull(Store#2376)], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/stores_data_set.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Store)], ReadSchema: struct<Store:int,Type:string>